In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calc_mutual_inductance(coil_1, coil_2, d, po=0, fi=0):
    mu0 = 4 * np.pi * 10 ** (-7)
    mutual_inductance = np.ones((len(coil_1), len(coil_2)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    for ri in range(len(coil_1)):
        for rj in range(len(coil_2)):
            M = 0
            for n in range(N):
                for k in range(K):
                    xk_xn = po + coil_1[ri] * np.cos(df2 * k) * np.cos(fi) - coil_2[rj] * np.cos(df1 * n)
                    yk_yn = coil_1[ri] * np.sin(df2 * k) * np.cos(fi) - coil_2[rj] * np.sin(df1 * n)
                    zk_zn = d + coil_1[ri] * np.cos(df2 * k) * np.sin(fi)
                    r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                    M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
            M *= mu0 * coil_1[ri] * coil_2[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    return np.sum(mutual_inductance)

def calc_self_inductance(coil, thin=0.001):
    mu0 = 4 * np.pi * 10 ** (-7)
    L = np.sum(mu0 * coil * (np.log(8 * coil / thin) - 7 / 4 + (thin ** 2) / (8 * coil ** 2) * (np.log(8 * coil / thin) + 1 / 3)))
    mutual_inductance = np.ones((len(coil), len(coil)))
    N = 90
    K = 90
    df1 = 2 * np.pi / N
    df2 = 2 * np.pi / K
    d = 0
    po = 0
    fi = 0
    for ri in range(len(coil)):
        for rj in range(len(coil)):
            M = 0
            if ri != rj:
                for n in range(N):
                    for k in range(K):
                        xk_xn = po + coil[ri] * np.cos(df2 * k) * np.cos(fi) - coil[rj] * np.cos(df1 * n)
                        yk_yn = coil[ri] * np.sin(df2 * k) * np.cos(fi) - coil[rj] * np.sin(df1 * n)
                        zk_zn = d + coil[ri] * np.cos(df2 * k) * np.sin(fi)
                        r12 = (xk_xn ** 2 + yk_yn ** 2 + zk_zn ** 2) ** 0.5
                        M += (np.cos(df2 * k - df1 * n) * df1 * df2) / r12
                M *= mu0 * coil[ri] * coil[rj] / (4 * np.pi)
            mutual_inductance[ri][rj] = M
    M = np.sum(mutual_inductance)
    L += M
    return L

def calc_coupling_coefficient(coil_1, coil_2, d, po=0, fi=0):
    M = calc_mutual_inductance(coil_1, coil_2, d, po=po, fi=fi)
    L1 = calc_self_inductance(coil_1)
    L2 = calc_self_inductance(coil_2)
    k = M / (L1 * L2) ** 0.5
    return k

def mutation(start, finish, x, r):
    x = np.random.uniform(low = start if x - r < start else x - r,
                          high = finish if x + r > finish else x + r)
    return x

# между витками одной катушки не должно быть пересечений
def test_coils(coil, thin):
    # между витками 2 и 3 катушки есть пересечение
    if coil[1] > coil[2]:
        if (coil[1] - coil[2] - 2 * thin) < 0:
            return False
    elif coil[2] > coil[1]:
        if (coil[2] - coil[1] - 2 * thin) < 0:
            return False
    return True

In [3]:
def adaptive_hill_climbing(coil_1, thin, d):
    good_mutation = []
    bad_mutation = []
    all_mutation = []
    
    start = 0.03 + 2 * thin
    finish = 0.09 - 2 * thin
    coil_2 = np.array([0.03, start, finish, 0.09])
    print("Начальное значение катушки:", coil_2)
    
    Q = 2
    band = 0.005
    thr_k = 1e-4
    
    p_m = np.random.random()
    i = 0 
    while True:
        i += 1
        fit_k = calc_coupling_coefficient(coil_1, coil_2, d)
        coil_2q = coil_2.copy()
        for q in range(Q):
            r = np.random.random()
            if r < p_m:
                coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
                # проверка на пересечение витков внутри катушки
                while not test_coils(coil_2q, thin):
                    print(f"Есть пересечения внутри катушки! {coil_2q}")
                    coil_2q[1 + q] = mutation(start, finish, coil_2q[1 + q], band)
        fit_kq = calc_coupling_coefficient(coil_1, coil_2q, d)
        if fit_kq > fit_k:
            fit_k = fit_kq
            coil_2 = coil_2q.copy()
            good_mutation.append((fit_kq, coil_2q))
        else:
            bad_mutation.append((fit_kq, coil_2q))
        all_mutation.append((fit_kq, coil_2q))
        print(f"iter: {i} k = {fit_kq}, катушка = {coil_2q}")
        if len(good_mutation) >= 2:
            if (good_mutation[-1][0] - good_mutation[-2][0]) / thr_k < 1:
                break
        if i == 100:
            print("Алгоритм сделал 100 итераций! Останавливаю алгоритм")
            break
    return good_mutation[-1][0], good_mutation[-1][1], len(good_mutation), len(bad_mutation), len(all_mutation)

In [4]:
info_ahc = []

coil_1 = np.linspace(0.028, 0.07, 4)
thin = 0.001
d = 0.005
N = 10

In [5]:
for i in range(N):
    print(f"{i} итерация запуска алгоритма adaptive hill climbing:", end='\n\n')
    info_ahc.append(adaptive_hill_climbing(coil_1, thin, d))

0 итерация запуска алгоритма adaptive hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.471778227315863, катушка = [0.03       0.0335939  0.08587613 0.09      ]
iter: 2 k = 0.47269997349547066, катушка = [0.03       0.03378951 0.08587613 0.09      ]
iter: 3 k = 0.4831354879474526, катушка = [0.03       0.03603783 0.08587613 0.09      ]
iter: 4 k = 0.5129600854509455, катушка = [0.03       0.03891375 0.08412135 0.09      ]
iter: 5 k = 0.48413379794097167, катушка = [0.03       0.03891375 0.08703451 0.09      ]
iter: 6 k = 0.480200775721204, катушка = [0.03       0.03891375 0.08732741 0.09      ]
iter: 7 k = 0.5159916592001963, катушка = [0.03       0.03843008 0.08347904 0.09      ]
iter: 8 k = 0.5198246898468145, катушка = [0.03       0.03393042 0.08028697 0.09      ]
iter: 9 k = 0.5198246898468145, катушка = [0.03       0.03393042 0.08028697 0.09      ]
iter: 10 k = 0.5363804513480617, катушка = [0.03       0.03733084 0.08028697 0.09      ]
iter: 11 k 

iter: 93 k = 0.6477412329171147, катушка = [0.03       0.05843026 0.06787969 0.09      ]
iter: 94 k = 0.6541761472429104, катушка = [0.03       0.05354202 0.06787969 0.09      ]
iter: 95 k = 0.6532194084014821, катушка = [0.03       0.05354202 0.06916096 0.09      ]
iter: 96 k = 0.6534788247712197, катушка = [0.03       0.05354202 0.06899061 0.09      ]
iter: 97 k = 0.6476490606093416, катушка = [0.03       0.05845348 0.06787969 0.09      ]
iter: 98 k = 0.6541761472429104, катушка = [0.03       0.05354202 0.06787969 0.09      ]
iter: 99 k = 0.6463152096295136, катушка = [0.03       0.05677913 0.07125414 0.09      ]
iter: 100 k = 0.6510699627933993, катушка = [0.03       0.05621457 0.06535008 0.09      ]
Алгоритм сделал 100 итераций! Останавливаю алгоритм
1 итерация запуска алгоритма adaptive hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.43699739587833447, катушка = [0.03  0.032 0.088 0.09 ]
iter: 2 k = 0.4750537214423233, катушка = [0.03       0.03

iter: 1 k = 0.43699739587833447, катушка = [0.03  0.032 0.088 0.09 ]
iter: 2 k = 0.43699739587833447, катушка = [0.03  0.032 0.088 0.09 ]
iter: 3 k = 0.43699739587833447, катушка = [0.03  0.032 0.088 0.09 ]
iter: 4 k = 0.4460150945998616, катушка = [0.03       0.03388688 0.088      0.09      ]
iter: 5 k = 0.4460150945998616, катушка = [0.03       0.03388688 0.088      0.09      ]
iter: 6 k = 0.46260119405546696, катушка = [0.03       0.03388688 0.0868718  0.09      ]
iter: 7 k = 0.46260119405546696, катушка = [0.03       0.03388688 0.0868718  0.09      ]
iter: 8 k = 0.46260119405546696, катушка = [0.03       0.03388688 0.0868718  0.09      ]
iter: 9 k = 0.47108144304354754, катушка = [0.03       0.03574576 0.0868718  0.09      ]
iter: 10 k = 0.47108144304354754, катушка = [0.03       0.03574576 0.0868718  0.09      ]
iter: 11 k = 0.47108144304354754, катушка = [0.03       0.03574576 0.0868718  0.09      ]
iter: 12 k = 0.47108144304354754, катушка = [0.03       0.03574576 0.0868718  0.0

iter: 94 k = 0.5995500297589044, катушка = [0.03       0.04138763 0.07494612 0.09      ]
iter: 95 k = 0.5995500297589044, катушка = [0.03       0.04138763 0.07494612 0.09      ]
iter: 96 k = 0.5995500297589044, катушка = [0.03       0.04138763 0.07494612 0.09      ]
iter: 97 k = 0.5848605977396831, катушка = [0.03       0.03808917 0.07494612 0.09      ]
iter: 98 k = 0.5995500297589044, катушка = [0.03       0.04138763 0.07494612 0.09      ]
iter: 99 k = 0.5802161794976591, катушка = [0.03       0.04138763 0.07721866 0.09      ]
iter: 100 k = 0.5819277793649894, катушка = [0.03       0.03749835 0.07494612 0.09      ]
Алгоритм сделал 100 итераций! Останавливаю алгоритм
3 итерация запуска алгоритма adaptive hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.46078475295434634, катушка = [0.03       0.03628981 0.08777797 0.09      ]
iter: 2 k = 0.5095772993947786, катушка = [0.03       0.03875036 0.08442241 0.09      ]
iter: 3 k = 0.4912209159965449, катушка

iter: 25 k = 0.6438743218218811, катушка = [0.03       0.05038634 0.0705123  0.09      ]
iter: 26 k = 0.6506271795258102, катушка = [0.03       0.05606858 0.064897   0.09      ]
iter: 27 k = 0.6544120054993184, катушка = [0.03       0.05403552 0.0686678  0.09      ]
iter: 28 k = 0.6456941619886207, катушка = [0.03       0.05768337 0.07093443 0.09      ]
iter: 29 k = 0.634334989969305, катушка = [0.03       0.05403552 0.07319648 0.09      ]
iter: 30 k = 0.6541867088075058, катушка = [0.03       0.05403552 0.06888957 0.09      ]
iter: 31 k = 0.6468824306823026, катушка = [0.03       0.04921149 0.06895212 0.09      ]
iter: 32 k = 0.6488154669138037, катушка = [0.03       0.05403552 0.07081005 0.09      ]
iter: 33 k = 0.6482805334394875, катушка = [0.03       0.04998209 0.0686678  0.09      ]
iter: 34 k = 0.6421078673154221, катушка = [0.03       0.05584568 0.0722015  0.09      ]
iter: 35 k = 0.6379758485517123, катушка = [0.03       0.05403552 0.0726794  0.09      ]
iter: 36 k = 0.6544423

iter: 83 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 84 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 85 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 86 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 87 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 88 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 89 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 90 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 91 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 92 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 93 k = 0.4469373186594012, катушка = [0.03       0.03409516 0.088      0.09      ]
iter: 94 k = 0.446937

iter: 89 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 90 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 91 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 92 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 93 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 94 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 95 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 96 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 97 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 98 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 99 k = 0.45455242533343004, катушка = [0.03       0.03581172 0.088      0.09      ]
iter: 100 

iter: 80 k = 0.6524056784668205, катушка = [0.03       0.05707167 0.0680493  0.09      ]
iter: 81 k = 0.6499643672168318, катушка = [0.03       0.05638174 0.06492205 0.09      ]
iter: 82 k = 0.6550510650816176, катушка = [0.03       0.05455995 0.0680493  0.09      ]
iter: 83 k = 0.6551445418281534, катушка = [0.03       0.05505424 0.0680493  0.09      ]
8 итерация запуска алгоритма adaptive hill climbing:

Начальное значение катушки: [0.03  0.032 0.088 0.09 ]
iter: 1 k = 0.43699739587833447, катушка = [0.03  0.032 0.088 0.09 ]
iter: 2 k = 0.4679745560021089, катушка = [0.03       0.03614579 0.08726109 0.09      ]
iter: 3 k = 0.499821488954532, катушка = [0.03       0.03614579 0.08407705 0.09      ]
iter: 4 k = 0.499821488954532, катушка = [0.03       0.03614579 0.08407705 0.09      ]
iter: 5 k = 0.499821488954532, катушка = [0.03       0.03614579 0.08407705 0.09      ]
iter: 6 k = 0.47448502090014105, катушка = [0.03       0.03614579 0.08674037 0.09      ]
iter: 7 k = 0.513515206286218

iter: 89 k = 0.6512193383678819, катушка = [0.03       0.05167181 0.06701839 0.09      ]
iter: 90 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 91 k = 0.6493832600813769, катушка = [0.03       0.05364614 0.0623623  0.09      ]
iter: 92 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 93 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 94 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 95 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 96 k = 0.6485094816563065, катушка = [0.03       0.04970935 0.06701839 0.09      ]
iter: 97 k = 0.6539525935973323, катушка = [0.03       0.05364614 0.06701839 0.09      ]
iter: 98 k = 0.6531818545234214, катушка = [0.03       0.05441735 0.06965132 0.09      ]
iter: 99 k = 0.6495045485346057, катушка = [0.03       0.05364614 0.06251317 0.09      ]
iter: 100 k = 0.65395

In [6]:
sorted(info_ahc, key=lambda x: x[0], reverse=True)

[(0.6551445418281534,
  array([0.03      , 0.05505424, 0.0680493 , 0.09      ]),
  17,
  66,
  83),
 (0.6544423644280375,
  array([0.03      , 0.05406707, 0.0686678 , 0.09      ]),
  14,
  22,
  36),
 (0.6541761472429104,
  array([0.03      , 0.05354202, 0.06787969, 0.09      ]),
  24,
  76,
  100),
 (0.6539525935973323,
  array([0.03      , 0.05364614, 0.06701839, 0.09      ]),
  15,
  85,
  100),
 (0.64855844991243,
  array([0.03      , 0.04445949, 0.058308  , 0.09      ]),
  18,
  35,
  53),
 (0.6484568072460906,
  array([0.03      , 0.04522982, 0.05810117, 0.09      ]),
  18,
  41,
  59),
 (0.5995500297589044,
  array([0.03      , 0.04138763, 0.07494612, 0.09      ]),
  14,
  86,
  100),
 (0.5691023837744105,
  array([0.03      , 0.04900584, 0.0802096 , 0.09      ]),
  14,
  68,
  82),
 (0.4681010179255878,
  array([0.03      , 0.03871844, 0.088     , 0.09      ]),
  3,
  97,
  100),
 (0.45455242533343004,
  array([0.03      , 0.03581172, 0.088     , 0.09      ]),
  1,
  99,
  100)

In [8]:
print(f"Лучший коэффициент связи {info_ahc[0][0]} у катушки {info_ahc[0][1]}")
print(f"Среднее количество итераций {sum([i[4] for i in info_ahc]) / len(info_ahc)}")
print(f"Среднее количество полезных мутаций {sum([i[2] for i in info_ahc]) / len(info_ahc)}")
print(f"Среднее количество бесполезных мутаций {sum([i[3] for i in info_ahc]) / len(info_ahc)}")

Лучший коэффициент связи 0.6541761472429104 у катушки [0.03       0.05354202 0.06787969 0.09      ]
Среднее количество итераций 81.3
Среднее количество полезных мутаций 13.8
Среднее количество бесполезных мутаций 67.5
